# Determine the [n, k, d] code configuration for a given polynomial A and B

#### Imports

In [1]:
import sys
import io
import numpy as np
import re
import time
from numpy.linalg import matrix_power as matrix_power
from numpy.linalg import matrix_rank as matrix_rank
from itertools import product

from mip import Model, xsum, minimize, BINARY
from bposd.css import css_code
from concurrent.futures import ThreadPoolExecutor

In [2]:
# Function to create the cyclic shift matrix S_l of size lxl
def get_cyclic_shift_matrix_S(l: int):
    S_l = np.roll(np.eye(l), 1, axis=1)
    return S_l

# Function to evaluate a polynomial at a matrix
def get_matrix_polynomial(poly, x, y):
    # Define a regular expression pattern to match 'x' or 'y' followed by numbers
    pattern = r'([xy])(\d*)'

    # Split the input string into individual terms
    terms = poly.split('+')

    # Initialize the result matrix as an identity matrix
    result_matrix = np.zeros_like(x)

    # Process each term separately
    for term in terms:
        # Find all matches in the term
        matches = re.findall(pattern, term)
        
        # Initialize the term result matrix as an identity matrix
        term_result_matrix = np.eye(x.shape[0])
        
        # Iterate through the matches and perform matrix multiplication with the corresponding exponents
        for match in matches:
            matrix_name = match[0]
            exponent = int(match[1]) if match[1] else 1
            
            if matrix_name == 'x':
                term_result_matrix = np.dot(term_result_matrix, matrix_power(x, exponent))
            elif matrix_name == 'y':
                term_result_matrix = np.dot(term_result_matrix, matrix_power(y, exponent))
        
        # Add the term result matrix to the overall result matrix
        result_matrix += term_result_matrix
    
    return result_matrix % 2

#### Choose Parameters

In [3]:
l = 8
m = 8

A_polynomial = 'x3 + y3 + y2 + y6'
B_polynomial = 'y3 + x3 + x2 + x6'

In [4]:
# Create the identity matrices I_ℓ and I_m
I_l = np.eye(l)
I_m = np.eye(m)

# Create the cyclic shift matrices S_ℓ and S_m
S_l = get_cyclic_shift_matrix_S(l)
S_m = get_cyclic_shift_matrix_S(m)

# Create matrices x and y based on the tensor product
x = np.kron(S_l, I_m)
y = np.kron(I_l, S_m)

A = get_matrix_polynomial(A_polynomial, x, y)
B = get_matrix_polynomial(B_polynomial, x, y)

##### Sanity Check

In [5]:
print(A_polynomial)


x3 + y3 + y2 + y6


In [6]:
print(B_polynomial)

y3 + x3 + x2 + x6


In [7]:
def commutator(A, B):
    return A @ B - B @ A

In [8]:
if not np.any(commutator(A, B)):
    print('A and B commute.')
else:
    raise ValueError('A and B do NOT commute as they should. Please check the code how they get created.')

A and B commute.


#### Generate Check-Matrices $H^{x} = [A|B]$ and $H^{z} = [B^{T} | A^{T}]$.

Here, the vertical bar indicates stacking the matrices horizontally and $T$ means the matrix transposition. As a check, both Hx and Hz should have the dimension $(n/2)$ x $n$

In [9]:
def generate_check_matrices(A, B):
    """
    Generate check matrices Hx and Hz.

    Parameters:
        A (numpy.ndarray): The matrix A.
        B (numpy.ndarray): The matrix B.

    Returns:
        Hx (numpy.ndarray): The check matrix Hx.
        Hz (numpy.ndarray): The check matrix Hz.
    """
    # Ensure that A and B have compatible dimensions
    if A.shape[0] != B.shape[0]:
        raise ValueError("A and B must have the same number of rows")

    # Calculate dimensions
    n = A.shape[1] + B.shape[1]

    # Create the check matrix Hx by horizontally stacking A and B
    Hx = np.hstack((A, B))

    # Create the check matrix Hz by horizontally stacking the transpose of B and A
    Hz = np.hstack((B.T, A.T))

    assert Hx.shape == (n/2, n) and Hz.shape == (n/2, n), "Hx and Hz must be of dimension (n/2, n)."
    return Hx, Hz

In [10]:
Hx, Hz = generate_check_matrices(A, B)

In [11]:
def check_commute(Hx, Hz):
    if not np.any((Hx @ Hz.T) % 2):
        print('X and Z checks commute.')
    else:
        raise ValueError('X and Z checks do NOT commute as they should. Please check the code how they get created.')

In [12]:
check_commute(Hx, Hz)

X and Z checks commute.


#### Obtain the code parameters $n, k, d$ of the **LDPC** code

##### Physical Data qubits $n$

In [13]:
n = 2*l*m
print('Number of PHYSICAL DATA qubits:', n)

Number of PHYSICAL DATA qubits: 128


##### Logical qubits $k$

Two approaches to calculate the number of logical qubits $k$:
- $k = n - \text{rank}(H^{x}) - \text{rank}(H^{z})$
- $k = 2 \cdot \text{dim}(\text{ker}(A) ∩ \text{ker}(B))$

Their equivalence is shown in Lemma 1 on page 10.

In [14]:
def binary_row_reduction(H):
    """
    Perform row reduction on a binary matrix H over the field F2.
    """
    # Copy of H to avoid modifying the original matrix
    R = H.copy()

    # Number of rows and columns
    rows, cols = R.shape

    # Row reduction
    row = 0
    for col in range(cols):
        if row >= rows:
            break

        # Find a row with a 1 in the current column
        for i in range(row, rows):
            if R[i, col] == 1:
                R[[row, i]] = R[[i, row]]  # Swap rows
                break
        else:
            continue

        # Eliminate 1s in the current column from other rows
        for i in range(rows):
            if i != row and R[i, col] == 1:
                R[i] = (R[i] + R[row]) % 2

        row += 1

    return R

def binary_matrix_rank(H):
    """
    Compute the rank of a binary matrix H over the field F2.
    """
    # Perform row reduction
    row_reduced_H = binary_row_reduction(H)

    # Count the number of non-zero rows
    rank = sum(np.any(row) for row in row_reduced_H)
    return rank

In [15]:
k = n - binary_matrix_rank(Hx) - binary_matrix_rank(Hz)
print('Number of LOGICAL qubits:', k)

Number of LOGICAL qubits: 16


#### Net encoding rate $r = \frac{k}{2n}$

In [16]:
def get_net_encoding_rate(k, n):
    return k / (2*n)

In [17]:
r = get_net_encoding_rate(k, n)
print('Net encoding rate:', r)

Net encoding rate: 0.0625


#### Using the MIP package

In [18]:
# computes the minimum Hamming weight of a binary vector x such that
# stab @ x = 0 mod 2
# logicOp @ x = 1 mod 2
# here stab is a binary matrix and logicOp is a binary vector
def distance_test(stab, logicOp):
  # number of qubits
  n = stab.shape[1]
  # number of stabilizers
  m = stab.shape[0]

  # maximum stabilizer weight
  wstab = np.max([np.sum(stab[i,:]) for i in range(m)])
  # weight of the logical operator
  wlog = np.count_nonzero(logicOp)
  # how many slack variables are needed to express orthogonality constraints modulo two
  num_anc_stab = int(np.ceil(np.log2(wstab)))
  num_anc_logical = int(np.ceil(np.log2(wlog)))
  # total number of variables
  num_var = n + m*num_anc_stab + num_anc_logical

  model = Model()
  model.verbose = 0
  x = [model.add_var(var_type=BINARY) for i in range(num_var)]
  model.objective = minimize(xsum(x[i] for i in range(n)))

  # orthogonality to rows of stab constraints
  for row in range(m):
    weight = [0]*num_var
    supp = np.nonzero(stab[row,:])[0]
    for q in supp:
      weight[q] = 1
    cnt = 1
    for q in range(num_anc_stab):
      weight[n + row*num_anc_stab +q] = -(1<<cnt)
      cnt+=1
    model+= xsum(weight[i] * x[i] for i in range(num_var)) == 0

  # odd overlap with logicOp constraint
  supp = np.nonzero(logicOp)[0]
  weight = [0]*num_var
  for q in supp:
    weight[q] = 1
  cnt = 1
  for q in range(num_anc_logical):
      weight[n + m*num_anc_stab +q] = -(1<<cnt)
      cnt+=1
  model+= xsum(weight[i] * x[i] for i in range(num_var)) == 1

  model.optimize()

  opt_val = sum([x[i].x for i in range(n)])

  return int(opt_val)


def distance_test_parallel(stab, logicOp):
    n = stab.shape[1]  # number of qubits
    m = stab.shape[0]  # number of stabilizers

    # Parallel computation of wstab
    with ThreadPoolExecutor() as executor:
        wstab_list = list(executor.map(np.sum, stab))
    wstab = np.max(wstab_list)

    wlog = np.count_nonzero(logicOp)
    num_anc_stab = int(np.ceil(np.log2(wstab)))
    num_anc_logical = int(np.ceil(np.log2(wlog)))
    num_var = n + m * num_anc_stab + num_anc_logical

    model = Model()
    model.verbose = 0
    x = [model.add_var(var_type=BINARY) for i in range(num_var)]
    model.objective = minimize(xsum(x[i] for i in range(n)))

    # Function to prepare and add a constraint for a row of stab
    def add_stab_constraint(row):
        weight = [0] * num_var
        supp = np.nonzero(stab[row, :])[0]
        for q in supp:
            weight[q] = 1
        cnt = 1
        for q in range(num_anc_stab):
            weight[n + row * num_anc_stab + q] = -(1 << cnt)
            cnt += 1
        return xsum(weight[i] * x[i] for i in range(num_var)) == 0

    # Parallel addition of orthogonality constraints
    with ThreadPoolExecutor() as executor:
        constraints = list(executor.map(add_stab_constraint, range(m)))
    for constraint in constraints:
        model += constraint

    # Adding odd overlap with logicOp constraint (not parallelized)
    supp = np.nonzero(logicOp)[0]
    weight = [0] * num_var
    for q in supp:
        weight[q] = 1
    cnt = 1
    for q in range(num_anc_logical):
        weight[n + m * num_anc_stab + q] = -(1 << cnt)
        cnt += 1
    model += xsum(weight[i] * x[i] for i in range(num_var)) == 1

    model.optimize()
    opt_val = sum(x[i].x for i in range(n))

    return int(opt_val)

#### Evaluation of a single code with a fixed set of parameters

In [42]:
# Code parameters
ell, m = 6, 6
a1, a2, a3, a4 = 4, 4, 2, 4
b1, b2, b3, b4 = 4, 4, 2, 4

n = 2*ell*m
n2 = ell*m

# define cyclic shift matrices
I_ell = np.identity(ell, dtype=int)
I_m = np.identity(m, dtype=int)
I = np.identity(ell*m, dtype=int)
x = {}
y = {}
for i in range(ell):
	x[i] = np.kron(np.roll(I_ell, i, axis=1), I_m)
for i in range(m):
	y[i] = np.kron(I_ell, np.roll(I_m, i, axis=1))

# define check matrices
A = (x[a1] + y[a2] + y[a3] + y[a4]) % 2
B = (y[b1] + x[b2] + x[b3] + x[b4]) % 2

AT = np.transpose(A)
BT = np.transpose(B)
hx = np.hstack((A, B))
hz = np.hstack((BT, AT))

qcode=css_code(hx,hz)
print('Testing CSS code...')
qcode.test()
print('Done')

lz = qcode.lz
lx = qcode.lx
k = lz.shape[0]

In [45]:
n_phys_qubits = qcode.N
n_log_qubits = qcode.K

print('Number of PHYSICAL DATA qubits:', n_phys_qubits)
print('Number of LOGICAL qubits:', n_log_qubits)

print('Net Encoding Rate:', get_net_encoding_rate(n_log_qubits, n_phys_qubits))

In [ ]:
start_time = time.time()

print('Computing code distance PARALLELIZED...')
# We compute the distance only for Z-type logical operators (the distance for X-type logical operators is the same)
# by solving an integer linear program (ILP). The ILP looks for a minimum weight Pauli Z-type operator which has an even overlap with each X-check
# and an odd overlap with logical-X operator on the i-th logical qubit. Let w_i be the optimal value of this ILP.
# Then the code distance for Z-type logical operators is dZ = min(w_1,…,w_k).
d = n
for i in range(k):
	w = distance_test_parallel(hx, lx[i,:])
	print('Logical qubit =', i+1, 'Distance =', w)
	d = min(d, w)

print(f'\nCode parameters: [n, k, d] = [{n}, {k}, {d}]')

print('Execution Time: ', round(time.time() - start_time, 1), 'seconds')

In [ ]:
start_time = time.time()
print('Computing code distance...')
# We compute the distance only for Z-type logical operators (the distance for X-type logical operators is the same)
# by solving an integer linear program (ILP). The ILP looks for a minimum weight Pauli Z-type operator which has an even overlap with each X-check
# and an odd overlap with logical-X operator on the i-th logical qubit. Let w_i be the optimal value of this ILP.
# Then the code distance for Z-type logical operators is dZ = min(w_1,…,w_k).
d = n
for i in range(k):
	w = distance_test_parallel(hx, lx[i,:])
	print('Logical qubit =', i+1, 'Distance =', w)
	d = min(d, w)

print(f'\nCode parameters: [n, k, d] = [{n}, {k}, {d}]')
print('Execution Time: ', round(time.time() - start_time, 1), 'seconds')

### Reproduction of codes in the paper

In [ ]:
l_m = [(6, 6), (15, 3), (9, 6), (12, 6), (12, 12), (30, 6), (21, 18)]

A_B_polynomials = [
    ('x3 + y + y2', 'y3 + x + x2'),
    ('x9 + y + y2', '1 + x2 + x7'),
    ('x3 + y + y2', 'y3 + x + x2'),
    ('x3 + y + y2', 'y3 + x + x2'),
    ('x3 + y2 + y7', 'y3 + x + x2'),
    ('x9 + y + y2', 'y3 + x25 + x26'),
    ('x3 + y10 + y17', 'y5 + x3 + x19')
]

In [ ]:
for ind, code_config in enumerate(A_B_polynomials):
    l, m  = l_m[ind]
    A_polynomial, B_polynomial = code_config

    # Create the identity matrices I_ℓ and I_m
    I_l = np.eye(l)
    I_m = np.eye(m)

    # Create the cyclic shift matrices S_ℓ and S_m
    S_l = get_cyclic_shift_matrix_S(l)
    S_m = get_cyclic_shift_matrix_S(m)

    # Create matrices x and y based on the tensor product
    x = np.kron(S_l, I_m)
    y = np.kron(I_l, S_m)

    A = get_matrix_polynomial(A_polynomial, x, y)
    B = get_matrix_polynomial(B_polynomial, x, y)

    Hx, Hz = generate_check_matrices(A, B)
    check_commute(Hx, Hz)

    # Calculate code parameters
    n = 2*l*m
    k = n - binary_matrix_rank(Hx) - binary_matrix_rank(Hz)

    print(f'Code Config l={l}, m={m}')
    print(f'[[n, k, d]]: [[{n}, {k}, d]]\n')